In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    


rawname_ = 'S01_off_hold'
#rawname_ = 'S01_on_hold'
rawname = rawname_ + '_resample_raw.fif'
fname_full = os.path.join(data_dir,rawname)
    
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)

In [ ]:
reconst_name = rawname_ + '_resample_afterICA_raw.fif'
reconst_fname_full = os.path.join(data_dir,reconst_name)
reconst_raw = mne.io.read_raw_fif(reconst_fname_full, None)

In [ ]:
raw

In [ ]:
#reconst_raw.ch_names

In [ ]:
#reshuffle channels types (by default LFP and EMG types are determined wronng)

# set types for some misc channels
for i,chn in enumerate(raw.ch_names):
    #chn = raw.ch_names[chi]
    show = 0
    if chn.find('_old') >= 0:
        raw.set_channel_types({chn:'emg'}); show = 1
    elif chn.find('_kil') >= 0:
        raw.set_channel_types({chn:'misc'}); show = 1
    elif chn.find('LFP') >= 0:
        raw.set_channel_types({chn:'bio'}); show = 1  # or stim, ecog, eeg
    
    if show:
        print(i, chn )
        
        
bt = mne.io.pick.channel_indices_by_type(raw.info)
miscchans = bt['misc']
gradchans = bt['grad']
magchans = bt['mag']
eogchans = bt['eog']
emgchans = bt['emg']
biochans = bt['bio']
#mne.pick_channels(raw,miscchans)

print('miscchans', len(miscchans))
print('gradchans', len(gradchans) )
print('magchans', len(magchans))
print('eogchans', len(eogchans))
print('emgchans', len(emgchans))
print('biochans', len(biochans))
print( len(miscchans) + len(gradchans) + len(magchans) + len(eogchans) + len(emgchans) +
      len(biochans), len(raw.ch_names) )
print(len(raw.info['bads']))

In [ ]:
# get info about bad MEG channels (from separate file)
with open('subj_info.json') as info_json:
        #raise TypeError

    #json.dumps({'value': numpy.int64(42)}, default=convert)
    gen_subj_info = json.load(info_json)
    
subj,medcond,task  = utils.getParamsFromRawname(rawname_)
badchlist = gen_subj_info[subj]['bad_channels'][medcond][task]
raw.info['bads'] = badchlist
print('bad channels are ',badchlist)

In [ ]:
import utils
trem_times_fn = 'trem_times_tau.json'
with open(trem_times_fn ) as jf:
    trem_times_byhand = json.load(jf)   
trem_times_nms_fn = 'trem_times_tau_nms.json'
with open(trem_times_nms_fn ) as jf:
    trem_times_nms_byhand = json.load(jf)   

#%debug
tremIntervalJan, artif         = utils.unpackTimeIntervals(trem_times_byhand, mainSide = True, 
                                                           gen_subj_info=gen_subj_info, skipNotLoadedRaws=0)
tremIntervalJan_nms, artif_nms = utils.unpackTimeIntervals(trem_times_nms_byhand, mainSide = False, 
                                                           gen_subj_info=gen_subj_info, skipNotLoadedRaws=0)
for rawn in [rawname_]:
    if rawn in artif_nms and rawn not in artif:
        artif[rawn] = artif_nms[rawn]
    else:
        if rawn in artif_nms:
            artif[rawn].update(artif_nms[rawn] )
        
for rawn in tremIntervalJan:
    sind_str,medcond,task = utils.getParamsFromRawname(rawn)
    maintremside = gen_subj_info[sind_str]['tremor_side']
    opside= utils.getOppositeSideStr(maintremside)
    if rawn in tremIntervalJan_nms:
        tremIntervalJan[rawn][opside] = tremIntervalJan_nms[rawn][opside] 


mvtTypes = ['tremor', 'no_tremor', 'unk_activity']

plotTremNegOffset = 2.
plotTremPosOffset = 2.
maxPlotLen = 6   # for those interval that are made for plotting, not touching intervals for stats
addIntLenStat = 5
plot_time_end = 150

timeIntervalPerRaw_processed = utils.processJanIntervals(tremIntervalJan, maxPlotLen, addIntLenStat, 
                          plotTremNegOffset, plotTremPosOffset, plot_time_end, mvtTypes=mvtTypes)

In [ ]:
# for current raw
maintremside = gen_subj_info[subj]['tremor_side']
nonmaintremside = utils.getOppositeSideStr(maintremside)
intervals = timeIntervalPerRaw_processed[rawname_][maintremside]   #[rawn][side] -- list of tuples (beg,end, type string)]   #[rawn][side] -- list of tuples (beg,end, type string)
intervals_nms = timeIntervalPerRaw_processed[rawname_][nonmaintremside]   #[rawn][side] -- list of tuples (beg,end, type string)]   #[rawn][side] -- list of tuples (beg,end, type string)

# convert to intervalType -> intervalInds
import globvars as gv
ivalis = {}  # dict of indices of intervals per interval type
ivalis_nms = {}
for itype in gv.gparams['intTypes']:
    ivit = []
    for i,interval in enumerate(intervals):
        t1,t2,it = interval

        if it == itype:
            ivit += [i]
    if len(ivit) > 0:
        ivalis[itype] = ivit
        
    ivit = []
    for i,interval in enumerate(intervals_nms):
        t1,t2,it = interval

        if it == itype:
            ivit += [i]
    if len(ivit) > 0:
        ivalis_nms[itype] = ivit

print('Main tremor side here is ',maintremside)

display('all intervals:' ,intervals)
display('intervals by type:', ivalis )

# convert intervals to MNE type
annotation_desc_2_event_id = {'middle_full':0, 'no_tremor':1, 'endseg':2}
annotation_desc_2_event_id = {'middle_full':0, 'no_tremor':1}

onset = [ ivl[0] for ivl in intervals ]
duration = [ ivl[1]-ivl[0] for ivl in intervals ]
description = [ ivl[2] for ivl in intervals ]
annot = mne.Annotations(onset, duration, description)
raw.set_annotations(annot)

In [ ]:
help(plt.hist)

In [ ]:
meg_chis = np.where ( [ a in ['grad','mag'] for a in reconst_raw.get_channel_types() ] )[0]

meg_chnames = set( np.array(reconst_raw.ch_names)[meg_chis] ) #- set( raw.info['bads'] )
meg_chnames = sorted(meg_chnames)

In [ ]:
reconst_raw.info['bads']

In [ ]:
raw.info['bads']

In [ ]:
meg_chnames

In [ ]:
#for chn in meg_chnames
iii = 8
chn = meg_chnames[iii]
#chn = raw.info['bads'][3]
print(chn)
chd, times = reconst_raw[chn]
chd0, times = raw[chn]

In [ ]:
chd = chd[0,:]
chd0 = chd0[0,:]

In [ ]:
ww = 5; hh = 3; nr = 2; nc = 1
fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr*hh), sharex='col'); axs = axs.reshape(2,1)
ax = axs[0,0]
ax.hist(chd0, bins=100, alpha=0.5);
ax = axs[1,0]
ax.hist(chd, bins=100, alpha=0.5);
ax.set_title('after Maxwell + filtering + ICA')

In [ ]:
raw.get_channel_types()

In [ ]:
lfp_chis = np.where ( [ a in ['bio'] for a in reconst_raw.get_channel_types() ] )[0]
lfp_chnames =  np.array(reconst_raw.ch_names)[lfp_chis] 

iii = 1
chn = lfp_chnames[iii]
print(chn)
chd, times = reconst_raw[chn]
chd0, times = raw[chn]

In [ ]:
chd = chd[0,:]
chd0 = chd0[0,:]

In [ ]:
ww = 5; hh = 3; nr = 2; nc = 1
fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr*hh), sharex='col'); axs = axs.reshape(2,1)
ax = axs[0,0]
ax.hist(chd0, bins=100, alpha=0.5);
ax = axs[1,0]
ax.hist(chd, bins=100, alpha=0.5);
ax.set_title('after filtering')

In [ ]:
strec = 0
endrec = 300
epdur = endrec
new_events = mne.make_fixed_length_events(reconst_raw, start=strec, stop=endrec, duration=epdur)
epochs = mne.Epochs(raw,new_events, tmin=0,tmax = epdur, baseline=None)

freqs = np.linspace(3, 90, num=100)

# sh=2
# cfq = 15
# freqs = np.logspace(*np.log10([cfq-sh,cfq+sh ]), num=6)
n_cycles = freqs / 2.  # different number of cycle per frequency
power = mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=False, decim=3, n_jobs=10)

### Plot power band concentrations

In [ ]:
timeints = [ ('nonotrem',a0,b0), ('trem', a,b), ('trem2', a1,b1)]
fbs = [ ('tremfreq', 3,9), ('alpha/mu', 8,13),  ('beta', 15,30), ('gamma', 30,90)  ]
nc = len(fbs); nr = len(timeints)
ww = 4; hh = 3
headsph = np.array([0,0,0,0.9])
fig,axs = plt.subplots( nrows = nr, ncols = nc, figsize= (nc*ww, nr*hh))
for i,ti in enumerate(timeints):
    tiname,timin,timax = ti
    for j,fb in enumerate(fbs):
        fbname, fbmin, fbmax = fb
        ax = axs[i,j]
        ttl = 'epoch type: {};  band: {}'.format(tiname,fbname)
        ax.set_title(ttl)
        power.plot_topomap(sensors=True, contours=8, tmin=timin, tmax=timax, 
                           fmin=fbmin, fmax=fbmax, axes=ax, colorbar=True, size=40, res=100, show=0, sphere=headsph);
        #plt.gcf().suptitle('{} : {}'.format(tiname,fbname))
#plt.tight_layout()
plt.savefig('{}_sensor_bandpow_concentr.pdf'.format(rawname_ ))

In [ ]:
reconst_raw.filter(picks='emg', l_freq=10, h_freq=None)

In [ ]:
#left
leftEMG, times = reconst_raw[['EMG063_old', 'EMG064_old']]
plt.plot(times,leftEMG.T, c='b', alpha=0.5)
plt.figure()
rightEMG, times = reconst_raw[['EMG061_old', 'EMG062_old']]
plt.plot(times,rightEMG.T, c='g', alpha=0.5)